# 모델 로드
lstm 모델 로드하기

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals
import pandas as pd
import numpy as np
import re
import csv
import tensorflow as tf
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

from tensorflow import keras

Using TensorFlow backend.


In [2]:
cols = ['sentiment','id','date','query_string','user','text']
tweets = pd.read_csv("./data/sentiment140_training.1600000.processed.noemoticon.csv", encoding = 'latin-1', header = None, names = cols)
tweets.isnull().values.any()

False

In [11]:
brand = '앱티브_2019kia'

In [12]:

#일반 데이터용.
file_cols = ['date', 'time', 'user', 'text']
file = pd.read_csv("./전처리/preprocessed/전처리최종_{}.csv".format(brand), encoding = 'utf-8', header = None, names = file_cols)


In [27]:
#전체 합본데이터용
file = pd.read_csv("./합본데이터결과/기아합본_외국어문헌제외토픽테이블.csv", encoding = 'utf-8', header = 0)
file.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,time,user,text,key,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,1,1,1,2018-02-08,23:37:59,Olafur_Tomasson,angry wish not buy kia last year !,2018-02-0823:37:59Olafur_Tomasson,1,7.0,0.5969,"[(0, 0.017873356), (1, 0.01786835), (2, 0.2957..."
1,2,2,2,2018-02-08,23:37:31,HoopVideos_NET,kia rookie ladder josh jackson enters mix ben ...,2018-02-0823:37:31HoopVideos_NET,2,5.0,0.6288,"[(2, 0.08842112), (5, 0.62881666), (7, 0.23793..."
2,3,3,3,2018-02-08,23:37:03,HillaryHubOLA,kia os name lili fox just find book class read...,2018-02-0823:37:03HillaryHubOLA,3,3.0,0.3863,"[(0, 0.08687749), (1, 0.10005543), (3, 0.38631..."
3,4,4,4,2018-02-08,23:36:51,HoopVideos_NET,kia rookie ladder josh jackson enters mix ben ...,2018-02-0823:36:51HoopVideos_NET,4,5.0,0.6288,"[(2, 0.08843764), (5, 0.6287973), (7, 0.237939..."
4,5,5,5,2018-02-08,23:36:36,RamknickL,just ! recently add kia sorento inventory . check,2018-02-0823:36:36RamknickL,5,5.0,0.6387,"[(0, 0.020846326), (1, 0.020849146), (2, 0.020..."


In [28]:
len(file)

94141

In [6]:
#Data Preprocessing
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [7]:
X=[]
sentences = list(tweets['text'])
for sen in sentences:
    X.append(preprocess_text(sen))
y = tweets['sentiment']

#sentiment를 긍정은 1, 부정은 0으로 수정
y = np.array(list(map(lambda x: 1 if x == 4 else 0, y)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#Preparing the Embedding Layer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 69

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [8]:
new_model = keras.models.load_model('./lstm모델/DB2048_twitter100D_69_30_lstm_model.h5', compile=False)
new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
new_model.summary()
loss, acc = new_model.evaluate(X_test, y_test, verbose=1)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 69, 100)           50785600  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 50,902,977
Trainable params: 117,377
Non-trainable params: 50,785,600
_________________________________________________________________
320000/320000 [==============================] - 267s 835us/sample - loss: 0.4027 - acc: 0.8178
복원된 모델의 정확도: 81.78%


# 모델 로드하고 여기서부터만 돌리기

In [29]:
instance = []
sentences = list(file['text'])
for sen in sentences:
    sen = str(sen)
    instance.append(preprocess_text(sen))
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(instance)
instance = tokenizer.texts_to_sequences(instance)

vocab_size = len(tokenizer.word_index) + 1
maxlen = 69

instance= pad_sequences(instance, padding='post', maxlen=maxlen)
results = new_model.predict(instance)
classes = new_model.predict_classes(instance)
print(type(results))
print(type(classes))

print(results)
print(classes)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[0.1980812 ]
 [0.94263864]
 [0.04509246]
 ...
 [0.93017447]
 [0.39690453]
 [0.44428813]]
[[0]
 [1]
 [0]
 ...
 [1]
 [0]
 [0]]


In [30]:
brand = '기아합본_외국어문헌제외토픽테이블'

In [21]:
with open('./감성결과/9일간감성결과/감성결과_{}.csv'.format(brand), 'w', encoding='utf-8') as f:
    for i in range(len(file)):
        f.write('{},{},{},{},{},{},{}\n'.format(i, file['date'][i], file['time'][i], file['user'][i], file['text'][i], results[i], classes[i]))

In [31]:
#합본데이터용
with open('./감성결과/9일간감성결과/감성결과2_{}.csv'.format(brand), 'w', encoding='utf-8') as f:
    for i in range(len(file)):
        f.write('{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(i, file['date'][i], file['time'][i], file['user'][i], file['text'][i], results[i], classes[i], file['key'][i],file['문서 번호'][i],file['가장 비중이 높은 토픽'][i],file['가장 높은 토픽의 비중'][i],file['각 토픽의 비중'][i]))

In [32]:
#합본데이터용
with open('./감성결과/9일간감성결과/감성결과3_{}.csv'.format(brand), 'w', encoding='utf-8') as f:
    for i in range(len(file)):
        f.write('{},{},{},{},{},{},{},{},{},{},{}\n'.format(i, file['date'][i], file['time'][i], file['user'][i], file['text'][i], results[i], classes[i], file['key'][i],file['문서 번호'][i],file['가장 비중이 높은 토픽'][i],file['가장 높은 토픽의 비중'][i]))

In [ ]:
위의꺼 cols = ['i','date', 'time', 'user', 'text', 'results', 'classes', 'key','문서 번호','가장 비중이 높은 토픽','가장 높은 토픽의 비중','각 토픽의 비중']